TODO: 

I) version base
- (done)  reprendre classifieur de chat hiérarchique, remplacer encodeur seq2vec par un seq2seq (stacked lstm eg)
    cf HierarchicalChatSequenceClassification
- (done)  ajouter séquence de labels / comme si arbre ou forêt (possible ds graph_parser ?)

- sousclasser graph_parser / modifs
- brancher données réelles

/home/muller/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/data/dataset_readers/semantic_dependency_parsing.py


améliorations: 
- encodeur tour -> bert
- graphe de labels -> AdjacencyMatrixField


In [17]:
import itertools
import logging
from typing import Dict, List, Iterable
import torch
import torch.optim as optim

In [18]:
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import Field, LabelField, TextField, ListField, SequenceLabelField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer, PretrainedTransformerTokenizer
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding

from allennlp.modules import Seq2VecEncoder, Seq2SeqEncoder
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.data.vocabulary import Vocabulary

from allennlp.training.trainer import Trainer
from allennlp.data.iterators import BucketIterator
from allennlp.common import Params

In [19]:
from dummy_chat_reader import ChatReader

In [20]:
token_indexers = {"tokens": SingleIdTokenIndexer()}

tokenizer_cfg = Params({"word_splitter": {"language": "en"}})

tokenizer = Tokenizer.from_params(tokenizer_cfg)


reader = ChatReader(
    tokenizer=tokenizer,
    token_indexers=token_indexers,
    )
train_instances = reader.read("./train_dummy.tsv")
vocab = Vocabulary.from_instances(train_instances)


for i in train_instances:
    print(i)
    i["lines"].index(vocab)
    i["label"].index(vocab)
    print(i["lines"].get_padding_lengths())

2it [00:00, 1341.96it/s]
100%|██████████| 2/2 [00:00<00:00, 27235.74it/s]

Instance with fields:
 	 lines: ListField of 3 TextFields : 
 	 TextField of length 7 with text: 
 		[this, is, a, turn, in, a, chat]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 2 with text: 
 		[another, one]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[but, this, is, different]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 
 	 label: SequenceLabelField of length 3 with labels:
 		['0', '0', '0']
 		in namespace: 'labels'. 

{'num_fields': 3, 'list_tokens_length': 7, 'list_num_tokens': 7}
Instance with fields:
 	 lines: ListField of 4 TextFields : 
 	 TextField of length 3 with text: 
 		[another, chat, starts]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[is, it, different, ?]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[well, a, little, bit]
 		and TokenIndexers : {'toke

In [21]:
print(i)

Instance with fields:
 	 lines: ListField of 4 TextFields : 
 	 TextField of length 3 with text: 
 		[another, chat, starts]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[is, it, different, ?]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 4 with text: 
 		[well, a, little, bit]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 TextField of length 9 with text: 
 		[but, we, need, a, different, number, of, turns, .]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 
 	 label: SequenceLabelField of length 4 with labels:
 		['0', '0', '1', '1']
 		in namespace: 'labels'. 



In [22]:
turn_encoder_cfg = Params({"type":"gru",'input_size': 100, 'hidden_size': 50, 'num_layers': 1,
                  'dropout': 0.25, 'bidirectional': False
})
#can be changed dynamically encoder_cfg["type"] = "lstm"
# warning: if bidirectional, state output dimension is hidden_size x 2 -> model doesn't know that

turn_encoder = Seq2VecEncoder.from_params(turn_encoder_cfg)
turn_encoder.hidden_size = turn_encoder_cfg["hidden_size"]


chat_encoder_cfg = Params({"type":"gru",'input_size': 50, 'hidden_size': 50, 'num_layers': 1,
                  'dropout': 0.25, 'bidirectional': False
})
chat_encoder = Seq2SeqEncoder.from_params(chat_encoder_cfg)
chat_encoder.hidden_size = chat_encoder_cfg["hidden_size"]



glove_text_field_embedder = Embedding.from_params(vocab,Params({"pretrained_file": "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.100d.txt.gz",
                                                          "embedding_dim": 100,
                                                          "trainable": False
}))

token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=100)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})



/home/muller/anaconda3/envs/allennlp/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
400000it [00:02, 157953.90it/s]


In [23]:
chat_encoder

PytorchSeq2SeqWrapper(
  (_module): GRU(50, 50, batch_first=True, dropout=0.25)
)

In [24]:
# from biaffine parser, another config (not used)
chat_encoder_cfg =  {
            "type": "stacked_bidirectional_lstm",
            "hidden_size": 400,
            "input_size": 200,
            "num_layers": 3,
            "recurrent_dropout_probability": 0.3,
            "use_highway": True
        }

In [25]:
from allennlp.models import Model
from typing import Dict, List, Iterable
from allennlp.modules import TimeDistributed
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits


class HierarchicalChatSequenceClassification(Model):

    # complicated way of doing it: better to put turn_encoder inside a curtom textfield embedder
    def __init__(self, vocab, embedder: TextFieldEmbedder, turn_encoder: Seq2VecEncoder, chat_encoder: Seq2VecEncoder):
        super(HierarchicalChatSequenceClassification, self).__init__(vocab)
        # turn encoder has to distribute over turns of a chat instance
        self.turn_encoder = TimeDistributed(turn_encoder)
        self.chat_encoder = chat_encoder
        self.text_embedder = embedder
        self.num_classes = self.vocab.get_vocab_size("label")
        self.tag_projection_layer = TimeDistributed(
            torch.nn.Linear(self.chat_encoder.get_output_dim(), self.num_classes)
        )
        
        self.accuracy = CategoricalAccuracy()
        self._loss = torch.nn.CrossEntropyLoss()
        
    def forward(self,
                lines,
                label = None):
        
        #breakpoint()
        # mask for each turn of each chat of the batch: shape = (batch_size x max_turns x tokens)
        mask = get_text_field_mask(lines,num_wrapping_dims=1)

        # chat turns fetching embedding
        # turns_embedding tensor is (batch_size x turns x max tokens x token embedding size)
        turns_embeddings = self.text_embedder(lines,num_wrapping_dims=1)
        
        # encoding turns
        # turn_h has shape (batch_size x turns x encoder_output_size) eg (1x3x50)
        turn_h = self.turn_encoder(turns_embeddings,mask)
        
        # encoding chat 
        # mask for chats is now nb of turns; beware weird return type of torch.max (tuple) 
        chat_mask = mask.max(axis=2)[0]
        # chat_h is all states for chat and has shape (batch_size x turns x encoder_output_size)
        chat_h = self.chat_encoder(turn_h,chat_mask)
        
        # the following is taken from simpletagger
        batch_size, sequence_length, _ = chat_h.size()
        # logits has shape (batchsize x turn nb x num classes)
        logits = self.tag_projection_layer(chat_h)
        reshaped_log_probs = logits.view(-1, self.num_classes)
        class_probabilities = torch.nn.functional.softmax(reshaped_log_probs, dim=-1).view(
            [batch_size, sequence_length, self.num_classes]
        )

        output_dict = {"logits": logits, "class_probabilities": class_probabilities}

        #breakpoint()
        if label is not None:
            tag_mask = chat_mask
            loss = sequence_cross_entropy_with_logits(logits, label, tag_mask)
            self.accuracy(logits, label, tag_mask)
            output_dict["loss"] = loss

        return output_dict


    # expected by trainer ??
    #def get_parameters_for_histogram_tensorboard_logging(self,*args,**kwargs):
    #    return []
    

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}



In [26]:
model = HierarchicalChatSequenceClassification(vocab,word_embeddings,turn_encoder,chat_encoder)

optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2,sorting_keys=[("lines","list_num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_instances,
                  should_log_parameter_statistics = False
)

In [27]:
trainer.train()

accuracy: 0.8571, loss: 0.6847 ||: 100%|██████████| 1/1 [00:00<00:00, 97.83it/s]
accuracy: 0.7143, loss: 0.6688 ||: 100%|██████████| 1/1 [00:00<00:00, 107.75it/s]
accuracy: 0.7143, loss: 0.6551 ||: 100%|██████████| 1/1 [00:00<00:00, 127.01it/s]
accuracy: 0.7143, loss: 0.6434 ||: 100%|██████████| 1/1 [00:00<00:00, 125.49it/s]
accuracy: 0.7143, loss: 0.6334 ||: 100%|██████████| 1/1 [00:00<00:00, 97.85it/s]
accuracy: 0.7143, loss: 0.6247 ||: 100%|██████████| 1/1 [00:00<00:00, 112.62it/s]
accuracy: 0.7143, loss: 0.6172 ||: 100%|██████████| 1/1 [00:00<00:00, 99.51it/s]
accuracy: 0.7143, loss: 0.6107 ||: 100%|██████████| 1/1 [00:00<00:00, 116.93it/s]
accuracy: 0.7143, loss: 0.6050 ||: 100%|██████████| 1/1 [00:00<00:00, 110.02it/s]
accuracy: 0.7143, loss: 0.6000 ||: 100%|██████████| 1/1 [00:00<00:00, 121.36it/s]
accuracy: 0.7143, loss: 0.5957 ||: 100%|██████████| 1/1 [00:00<00:00, 85.54it/s]
accuracy: 0.7143, loss: 0.5918 ||: 100%|██████████| 1/1 [00:00<00:00, 92.20it/s]
accuracy: 0.7143, los

{'best_epoch': 19,
 'peak_cpu_memory_MB': 334.952,
 'peak_gpu_0_memory_MB': 959,
 'training_duration': '0:00:00.594361',
 'training_start_epoch': 0,
 'training_epochs': 19,
 'epoch': 19,
 'training_accuracy': 0.7142857142857143,
 'training_loss': 0.5723774433135986,
 'training_cpu_memory_MB': 334.952,
 'training_gpu_0_memory_MB': 959}

In [16]:
vocab

Vocabulary with namespaces:  tokens, Size: 24 || labels, Size: 2 || Non Padded Namespaces: {'*tags', '*labels'}